In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time
import pymongo
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\skdab\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\skdab\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


# NASA Mars News

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [4]:
# Find the latest news title and paragraph 
slides = soup.find_all('li', class_='slide')
content_title = slides[0].find('div', class_ = 'content_title')
news_title = content_title.text.strip()
article_teaser_body = slides[0].find('div', class_ = 'article_teaser_body')
news_p = article_teaser_body.text.strip()
print("Title: ",news_title)
print("Paragraph: ",news_p)

Title:  NASA's Perseverance Scouts Mars Sample Return Campaign Landing Sites
Paragraph:  The six-wheeled explorer has inspected a stretch of the Red Planet to see if it is flat enough for NASA’s next Mars lander.


# JPL Mars Space Images-Featured Image

In [19]:
# Navigate to the page
img_url = 'https://spaceimages-mars.com'
browser.visit(img_url)

In [20]:
# HTML object
html=browser.html
# Parse HTML
soup=bs(html,"html.parser")
# Retrieve image url
result=soup.find('img',class_='headerimage fade-in')

In [21]:
print(result)

<img class="headerimage fade-in" src="image/featured/mars2.jpg"/>


In [27]:
image_url=result['src']

In [28]:
print(image_url)

image/featured/mars2.jpg


In [32]:
featured_image_url=img_url+'/'+image_url


In [33]:
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg
